In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from pulp import *

In [11]:
df=pd.read_excel(r"C:\Users\vinit\Downloads\Travelling_3x3.xlsx",index_col=0)
df

,Unnamed: 1,Bihar,Bihar.1,Bihar.2,Delhi,Delhi.1,Delhi.2,Pune,Pune.1,Pune.2,Kolkata,Kolkata.1,Kolkata.2,Inventory
NaN,NaN,M1,M2,M3,M1,M2,M3,M1,M2,M3,M1,M2,M3,NaN
Mumbai,M1,12,100000,100000,5,100000,100000,7,100000,100000,4,100000,100000,50.0
Mumbai,M2,100000,12,100000,100000,5,100000,100000,7,100000,100000,4,100000,40.0
Mumbai,M3,100000,100000,12,100000,100000,5,100000,100000,7,100000,100000,4,40.0
Hyderabad,M1,6,100000,100000,8,100000,100000,4,0,100000,12,100000,100000,35.0
Hyderabad,M2,100000,6,100000,100000,8,100000,100000,4,100000,100000,12,100000,58.0
Hyderabad,M3,100000,100000,6,100000,100000,8,100000,100000,4,100000,100000,12,45.0
Madhya Pradesh,M1,6,100000,100000,5,100000,100000,2,100000,100000,4,100000,100000,75.0
Madhya Pradesh,M2,100000,6,100000,100000,5,100000,100000,2,100000,100000,4,100000,72.0
Madhya Pradesh,M3,100000,100000,6,100000,100000,5,100000,100000,2,100000,100000,4,45.0


In [16]:
supplier_data = df.iloc[1:-1, 1:-1]
supplier_inventory = df.iloc[1:-1,-5:-4]
supplier_demand = df.iloc[-1:, 1:-5]
supplier_data_arr = supplier_data.values
supplier_inventory_arr = supplier_inventory.values
supplier_demand_arr = supplier_demand.values

In [17]:
supplier_data

,Bihar,Bihar.1,Bihar.2,Delhi,Delhi.1,Delhi.2,Pune,Pune.1,Pune.2,Kolkata,Kolkata.1,Kolkata.2
Mumbai,12,100000,100000,5,100000,100000,7,100000,100000,4,100000,100000
Mumbai,100000,12,100000,100000,5,100000,100000,7,100000,100000,4,100000
Mumbai,100000,100000,12,100000,100000,5,100000,100000,7,100000,100000,4
Hyderabad,6,100000,100000,8,100000,100000,4,0,100000,12,100000,100000
Hyderabad,100000,6,100000,100000,8,100000,100000,4,100000,100000,12,100000
Hyderabad,100000,100000,6,100000,100000,8,100000,100000,4,100000,100000,12
Madhya Pradesh,6,100000,100000,5,100000,100000,2,100000,100000,4,100000,100000
Madhya Pradesh,100000,6,100000,100000,5,100000,100000,2,100000,100000,4,100000
Madhya Pradesh,100000,100000,6,100000,100000,5,100000,100000,2,100000,100000,4
Telangana,8,100000,100000,10,100000,100000,2,100000,100000,8,100000,100000


In [14]:
supplier_inventory_arr

array([[100000],
       [100000],
       [7],
       [100000],
       [100000],
       [4],
       [100000],
       [100000],
       [2],
       [100000],
       [100000],
       [2]], dtype=object)

In [9]:
supplier_demand_arr

array([[40, 30, 52, 30, 40, 38, 48, 52, 30, 52, 38, 40]], dtype=object)

In [10]:
prob = LpProblem("TransportationProblem", LpMinimize)

In [14]:
c = supplier_data_arr
s = supplier_inventory_arr
d = supplier_demand_arr
rows = len(c)
cols =len(c[0])
x = [[LpVariable(f"x{i}_{j}", lowBound=0) for j in range(cols)] for i in range(rows)]
prob += lpSum(c[i][j] * x[i][j] for i in range(rows) for j in range(cols))
# Define supplier constrain
for i in range(rows):
    prob += lpSum(x[i][j] for j in range(cols)) <= s[i]
#Define Demand Constrain
# for j in range(cols):
#     prob += lpSum(x[i][j] for i in range(rows)) >= d[j]
# Define Demand Constraint
for k in range(cols):  # Loop over columns in demand array
    prob += lpSum(x[i][k] for i in range(rows)) >= d[0][k]  # Access the correct element in the demand array

prob.solve()
# Print the results
# print("Status for Material_1:", prob.status)
# print("Optimal Solution for Material_1:")
# print()
# for i in range(rows):
#     for j in range(cols):
#         print(f"{supplier[i]} to {cities[j]} = {x[i][j][0].varValue}")
# print(f"Total Cost for Material_1 = {round(prob.objective.value(), 2)}")

# Print the results
for k in range(cols):  # Loop over columns in demand array
    print(f"Status for Material_{k+1}:", prob.status)
    print(f"Optimal Solution for Material_{k+1}:")
    print()
    for i in range(rows):
        for j in range(cols):
            print(f"{supplier[i]} to {cities[j]} = {x[i][j][k].varValue}")
    print(f"Total Cost for Material_{k+1} = {round(prob.objective.value(), 2)}")
    print()


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


PulpSolverError: Pulp: Error while executing C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulp\solverdir\cbc\win\64\cbc.exe

In [12]:
print (x)

[[x0_0, x0_1, x0_2, x0_3, x0_4, x0_5, x0_6, x0_7, x0_8, x0_9, x0_10, x0_11], [x1_0, x1_1, x1_2, x1_3, x1_4, x1_5, x1_6, x1_7, x1_8, x1_9, x1_10, x1_11], [x2_0, x2_1, x2_2, x2_3, x2_4, x2_5, x2_6, x2_7, x2_8, x2_9, x2_10, x2_11], [x3_0, x3_1, x3_2, x3_3, x3_4, x3_5, x3_6, x3_7, x3_8, x3_9, x3_10, x3_11], [x4_0, x4_1, x4_2, x4_3, x4_4, x4_5, x4_6, x4_7, x4_8, x4_9, x4_10, x4_11], [x5_0, x5_1, x5_2, x5_3, x5_4, x5_5, x5_6, x5_7, x5_8, x5_9, x5_10, x5_11], [x6_0, x6_1, x6_2, x6_3, x6_4, x6_5, x6_6, x6_7, x6_8, x6_9, x6_10, x6_11], [x7_0, x7_1, x7_2, x7_3, x7_4, x7_5, x7_6, x7_7, x7_8, x7_9, x7_10, x7_11], [x8_0, x8_1, x8_2, x8_3, x8_4, x8_5, x8_6, x8_7, x8_8, x8_9, x8_10, x8_11], [x9_0, x9_1, x9_2, x9_3, x9_4, x9_5, x9_6, x9_7, x9_8, x9_9, x9_10, x9_11], [x10_0, x10_1, x10_2, x10_3, x10_4, x10_5, x10_6, x10_7, x10_8, x10_9, x10_10, x10_11], [x11_0, x11_1, x11_2, x11_3, x11_4, x11_5, x11_6, x11_7, x11_8, x11_9, x11_10, x11_11]]


In [13]:
print (prob)

TransportationProblem:
MINIMIZE
12*x0_0 + 100000*x0_1 + 100000*x0_10 + 100000*x0_11 + 100000*x0_2 + 5*x0_3 + 100000*x0_4 + 100000*x0_5 + 7*x0_6 + 100000*x0_7 + 100000*x0_8 + 4*x0_9 + 100000*x10_0 + 8*x10_1 + 8*x10_10 + 100000*x10_11 + 100000*x10_2 + 100000*x10_3 + 10*x10_4 + 100000*x10_5 + 100000*x10_6 + 2*x10_7 + 100000*x10_8 + 100000*x10_9 + 100000*x11_0 + 100000*x11_1 + 100000*x11_10 + 8*x11_11 + 8*x11_2 + 100000*x11_3 + 100000*x11_4 + 10*x11_5 + 100000*x11_6 + 100000*x11_7 + 2*x11_8 + 100000*x11_9 + 100000*x1_0 + 12*x1_1 + 4*x1_10 + 100000*x1_11 + 100000*x1_2 + 100000*x1_3 + 5*x1_4 + 100000*x1_5 + 100000*x1_6 + 7*x1_7 + 100000*x1_8 + 100000*x1_9 + 100000*x2_0 + 100000*x2_1 + 100000*x2_10 + 4*x2_11 + 12*x2_2 + 100000*x2_3 + 100000*x2_4 + 5*x2_5 + 100000*x2_6 + 100000*x2_7 + 7*x2_8 + 100000*x2_9 + 6*x3_0 + 100000*x3_1 + 100000*x3_10 + 100000*x3_11 + 100000*x3_2 + 8*x3_3 + 100000*x3_4 + 100000*x3_5 + 4*x3_6 + 100000*x3_8 + 12*x3_9 + 100000*x4_0 + 6*x4_1 + 12*x4_10 + 100000*x4_11 + 100

In [77]:
# New Code 
#New 
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Read the input data
data = pd.read_excel(r"C:\Users\User\Downloads\archive\Data_3val.xlsx", index_col=0)  # Replace with the actual path to your Excel file
df = data.copy()
# print("hiiiii")
# Extract data from the DataFrame
# supply_cities = df.iloc[:-1, 1:-1]  # Exclude the last row (Demand) and the first column (Supply cities)
# demand_cities = df.iloc[-1, 1:-1]
# materials = df.columns[1:-1]

supply_cities = df.iloc[1:-1, 1:-4]
demand_cities = df.iloc[-1:,1:-5]
materials = df.iloc[1:-1,-5:-4]
supply_cities_arr = supply_cities.values
# print(supply_cities_arr)
# print(type(supply_cities_arr))
demand_cities_arr = demand_cities.values
# print(demand_cities_arr)
print(type(demand_cities_arr))
materials_arr = materials.values
# print("Wwwwww")
# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)
# print(prob)
# Define decision variables
supply_cities_tuples = [tuple(row) for row in supply_cities_arr]
# print("sssssssssssssssssssssssssssssssssssssssssssssss",supply_cities_tuples)
demand_cities_tuples = [tuple(row) for row in demand_cities_arr]
# print("ddddddddddddddddddddddddddddddddddd",demand_cities_tuples)
materials_tuples = [tuple(row) for row in materials_arr]
# print("mmmmmmmmmmmmmmmmmmmmmmmmmmmm",materials_tuples)
x = LpVariable.dicts("x", ((i, j, k) for i in supply_cities_tuples for j in demand_cities_tuples for k in materials_tuples), lowBound=0)
# print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaam",x)



# Define decision variables
# x = LpVariable.dicts("x", ((i, j, k) for i in supply_cities_arr for j in demand_cities_arr for k in materials_arr), lowBound=0)

# Define the objective function
# prob += lpSum(df.loc[i, j] * x[i, j, k] for i in supply_cities_arr for j in demand_cities_arr for k in materials_arr)
# Define the objective function
# prob += lpSum(df.loc[i, j] * x[i, j, k] for i, row_i in supply_cities.iterrows() for j, row_j in demand_cities.iterrows() for k in materials_arr)
for i,row_i in supply_cities.iterrows():
    for j,row_j in demand_cities.iterrows():
        for k in materials_arr:
            print ("This is Supply_Cities",i,row_i)
            print ("This is Demand_cities",j,row_i)
            print ("This is material_arr",k)
# print("prrrrr",prob)


# Define supply constraints
# for i in supply_cities_arr:
#     for k in materials_arr:
#         prob += lpSum(x[i, j, k] for j in demand_cities_arr) <= df.loc[i, f"Inventory_{k}"]
# for i in supply_cities_arr:
#     for k in materials_arr:
#         for j in demand_cities_arr:
#             prob += lpSum(x[i,j,k]) <= 
print("---------------------------------------------------------------")
for i in supply_cities_arr:
    for k in materials_arr:
        
        # Convert i to a tuple
        i_tuple = tuple(i)
        
        print("ggggggggggg",i_tuple)
        
        # Convert demand_cities_arr to a list of tuples
        demand_cities_tuples = [tuple(row) for row in demand_cities_arr]
        
        # Define a constraint using lpSum
        # constraint_expression = lpSum(x[i_tuple, tuple(j), k] for j in demand_cities_tuples)
        for j in demand_cities_tuples:
            # print("Jinnnnnnnnnnn",j)
            constraint_expression = lpSum(x[i_tuple,j ,k]for k in materials_tuples)
            # print (constraint_expression)

        # Retrieve the inventory limit for material k in supply city i
        # inventory_limit = df.loc[i, f"Inventory_{k}"]
        inventory_key = f"Inventory_{str(k)}"
        # cell_key = f"Cities_{i[-2]}"
        # print ("Cell_Keyyyyyyyy",cell_key)
        # print ("Key of Inventory",inventory_key)
        inventory_limit = df.loc[i, inventory_key]


        # Add the constraint to the linear programming problem
        prob += constraint_expression <= inventory_limit

        # print("welcomeeeee",prob)
# Define demand constraints
for j in demand_cities_arr:
    print ("Jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj",j)
    prob += lpSum(x[i, j, k] for i in supply_cities_arr for k in materials_arr) == demand_cities_arr[j]
    

# Solve the problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Optimal Solution:")
for i in supply_cities.index:
    for j in demand_cities.index:
        for k in materials:
            if x[i, j, k].varValue > 0:
                print(f"{i} to {j} ({k}) = {x[i, j, k].varValue}")

print("\nTotal Cost =", round(prob.objective.value(), 2))

<class 'numpy.ndarray'>
This is Supply_Cities Mumbai Bihar            12
Bihar.1      100000
Bihar.2      100000
Delhi             5
Delhi.1      100000
Delhi.2      100000
Pune              7
Pune.1       100000
Pune.2       100000
Kolkata           4
Kolkata.1    100000
Kolkata.2    100000
Inventory      50.0
Name: Mumbai, dtype: object
This is Demand_cities Demand Bihar            12
Bihar.1      100000
Bihar.2      100000
Delhi             5
Delhi.1      100000
Delhi.2      100000
Pune              7
Pune.1       100000
Pune.2       100000
Kolkata           4
Kolkata.1    100000
Kolkata.2    100000
Inventory      50.0
Name: Mumbai, dtype: object
This is material_arr [50.]
This is Supply_Cities Mumbai Bihar            12
Bihar.1      100000
Bihar.2      100000
Delhi             5
Delhi.1      100000
Delhi.2      100000
Pune              7
Pune.1       100000
Pune.2       100000
Kolkata           4
Kolkata.1    100000
Kolkata.2    100000
Inventory      50.0
Name: Mumbai, dtype: objec

KeyError: 'Inventory_[50.]'

In [20]:
#Try_12_

import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Read the input data
data = pd.read_excel(r"C:\Users\vinit\Downloads\Travelling_3x3.xlsx", index_col=0)

# Extract data from the DataFrame
supply_cities = data.iloc[1:-1, 1:-1]
demand_cities = data.iloc[-1:, 1:-1]
inventory = data.iloc[1:-1, -1].values
demand = data.iloc[-1, 1:-1].values
costs = data.iloc[1:-1, 1:-1]

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define decision variables
x = LpVariable.dicts("x", ((i, j) for i in supply_cities.index for j in demand_cities.columns), lowBound=0)


# Define the objective function
prob += lpSum(costs.at[i, j] * x[(i, j)] for i in supply_cities.index for j in demand_cities.columns)
# Define supply constraints
# for i, inv in enumerate(inventory):
#     prob += lpSum(x[(i, j)] for j in demand_cities.columns) <= inv
# Define supply constraints
for i, inv in enumerate(inventory):
    prob += lpSum(x[(supply_cities.index[i], j)] for j in demand_cities.columns) <= inv


# Define demand constraints
for j in demand_cities.columns:
    prob += lpSum(x[(i, j)] for i in supply_cities.index) >= demand_cities[j].iloc[0]

# Solve the problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Optimal Solution:")
for i in supply_cities.index:
    for j in demand_cities.columns:
        if x[(i, j)].varValue > 0:
            print(f"{i} to {j} = {x[(i, j)].varValue}")

print("\nTotal Cost =", round(prob.objective.value(), 2))


Status: -1
Optimal Solution:
Mumbai to Delhi.2 = 12.666667
Mumbai to Kolkata = 17.333333
Mumbai to Kolkata.1 = 12.666667
Mumbai to Delhi.2 = 12.666667
Mumbai to Kolkata = 17.333333
Mumbai to Kolkata.1 = 12.666667
Mumbai to Delhi.2 = 12.666667
Mumbai to Kolkata = 17.333333
Mumbai to Kolkata.1 = 12.666667
Hyderabad to Bihar = 13.333333
Hyderabad to Bihar.1 = 10.0
Hyderabad to Pune.1 = 17.333333
Hyderabad to Bihar = 13.333333
Hyderabad to Bihar.1 = 10.0
Hyderabad to Pune.1 = 17.333333
Hyderabad to Bihar = 13.333333
Hyderabad to Bihar.1 = 10.0
Hyderabad to Pune.1 = 17.333333
Madhya Pradesh to Bihar.2 = 5.6666667
Madhya Pradesh to Delhi = 12.666667
Madhya Pradesh to Delhi.1 = 13.333333
Madhya Pradesh to Kolkata.2 = 13.333333
Madhya Pradesh to Bihar.2 = 5.6666667
Madhya Pradesh to Delhi = 12.666667
Madhya Pradesh to Delhi.1 = 13.333333
Madhya Pradesh to Kolkata.2 = 13.333333
Madhya Pradesh to Bihar.2 = 5.6666667
Madhya Pradesh to Delhi = 12.666667
Madhya Pradesh to Delhi.1 = 13.333333
Madhya

In [134]:
print (x)
print (prob)

{('Mumbai', 'Bihar'): x_('Mumbai',_'Bihar'), ('Mumbai', 'Bihar.1'): x_('Mumbai',_'Bihar.1'), ('Mumbai', 'Bihar.2'): x_('Mumbai',_'Bihar.2'), ('Mumbai', 'Delhi'): x_('Mumbai',_'Delhi'), ('Mumbai', 'Delhi.1'): x_('Mumbai',_'Delhi.1'), ('Mumbai', 'Delhi.2'): x_('Mumbai',_'Delhi.2'), ('Mumbai', 'Pune'): x_('Mumbai',_'Pune'), ('Mumbai', 'Pune.1'): x_('Mumbai',_'Pune.1'), ('Mumbai', 'Pune.2'): x_('Mumbai',_'Pune.2'), ('Mumbai', 'Kolkata'): x_('Mumbai',_'Kolkata'), ('Mumbai', 'Kolkata.1'): x_('Mumbai',_'Kolkata.1'), ('Mumbai', 'Kolkata.2'): x_('Mumbai',_'Kolkata.2'), ('Hyderabad', 'Bihar'): x_('Hyderabad',_'Bihar'), ('Hyderabad', 'Bihar.1'): x_('Hyderabad',_'Bihar.1'), ('Hyderabad', 'Bihar.2'): x_('Hyderabad',_'Bihar.2'), ('Hyderabad', 'Delhi'): x_('Hyderabad',_'Delhi'), ('Hyderabad', 'Delhi.1'): x_('Hyderabad',_'Delhi.1'), ('Hyderabad', 'Delhi.2'): x_('Hyderabad',_'Delhi.2'), ('Hyderabad', 'Pune'): x_('Hyderabad',_'Pune'), ('Hyderabad', 'Pune.1'): x_('Hyderabad',_'Pune.1'), ('Hyderabad', 'Pu

# Attempt_13

In [42]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Sample data in tabular form
# data = {
#     '': ['Mumbai', 'Mumbai', 'Mumbai', 'Hyderabad', 'Hyderabad', 'Hyderabad', 'Madhya Pradesh', 'Madhya Pradesh', 'Madhya Pradesh', 'Telangana', 'Telangana', 'Telangana', 'Demand'],
#     'Bihar_M1': [12, 100000, 100000, 6, 100000, 100000, 6, 100000, 100000, 8, 100000, 100000, 40],
#     'Bihar_M2': [100000, 12, 100000, 6, 100000, 100000, 6, 100000, 100000, 8, 100000, 100000, 30],
#     'Bihar_M3': [100000, 100000, 12, 6, 100000, 100000, 5, 100000, 100000, 8, 100000, 100000, 52],
#     'Delhi_M1': [5, 100000, 100000, 8, 100000, 100000, 5, 100000, 100000, 10, 100000, 100000, 30],
#     'Delhi_M2': [100000, 5, 100000, 8, 100000, 100000, 5, 100000, 100000, 10, 100000, 100000, 40],
#     'Delhi_M3': [100000, 100000, 5, 8, 100000, 100000, 2, 100000, 100000, 2, 100000, 100000, 38],
#     'Pune_M1': [7, 100000, 100000, 4, 0, 100000, 4, 100000, 100000, 8, 100000, 100000, 48],
#     'Pune_M2': [100000, 7, 100000, 4, 100000, 100000, 100000, 2, 100000, 100000, 2, 100000, 52],
#     'Pune_M3': [100000, 100000, 7, 4, 100000, 100000, 100000, 100000, 2, 100000, 100000, 8, 30],
# }
data = pd.read_excel(r"C:\Users\vinit\Downloads\Travelling_3x3.xlsx", index_col=0)
df = data.copy()

# df = pd.DataFrame(data)
# df.set_index('', inplace=True)

# Extracting supplier data, inventory, and demand
supplier_data = df.iloc[1:-1, 1:-1]
supplier_inventory = df.iloc[1:-1,-1]
supplier_demand = df.iloc[-1, 1:-1]
#
# supply_cities = data.iloc[1:-1, 1:-1]
# demand_cities = data.iloc[-1:, 1:-1]
# inventory = data.iloc[1:-1, -1].values
# demand = data.iloc[-1, 1:-1].values
# costs = data.iloc[1:-1, 1:-1]
# arr_data = supplier_data.values
# arr_inventory = supplier_inventory.values
# arr_demand = supplier_demand.values

arr_data = supplier_data.values
arr_inventory = supplier_inventory.values
arr_demand = supplier_demand.values

supply = supplier_data.index.tolist()
city = df.columns.tolist()[:-1]

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define decision variables
rows = len(arr_data)
cols = len(arr_data[0])

x = [[LpVariable(f"x{i}_{j}", lowBound=0) for j in range(cols)] for i in range(rows)]

# Define the objective function
prob += lpSum(arr_data[i][j] * x[i][j] for i in range(rows) for j in range(cols))

# Define supply constraints
for i in range(rows):
    prob += lpSum(x[i][j] for j in range(cols)) <= arr_inventory[i]

# Define demand constraints
for j in range(cols):
    prob += lpSum(x[i][j] for i in range(rows)) >= arr_demand[j]

# Solve the problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Optimal Solution:")
for i in range(rows):
    for j in range(cols):
        print(f"{supply[i]}_{city[j]} = {x[i][j].varValue}")
print ()
print("Total Cost = ", round(prob.objective.value(), 2))


Status: 1
Optimal Solution:
Mumbai_Unnamed: 1 = 0.0
Mumbai_Bihar = 0.0
Mumbai_Bihar.1 = 0.0
Mumbai_Bihar.2 = 30.0
Mumbai_Delhi = 0.0
Mumbai_Delhi.1 = 0.0
Mumbai_Delhi.2 = 0.0
Mumbai_Pune = 0.0
Mumbai_Pune.1 = 0.0
Mumbai_Pune.2 = 20.0
Mumbai_Kolkata = 0.0
Mumbai_Kolkata.1 = 0.0
Mumbai_Unnamed: 1 = 0.0
Mumbai_Bihar = 0.0
Mumbai_Bihar.1 = 0.0
Mumbai_Bihar.2 = 0.0
Mumbai_Delhi = 40.0
Mumbai_Delhi.1 = 0.0
Mumbai_Delhi.2 = 0.0
Mumbai_Pune = 0.0
Mumbai_Pune.1 = 0.0
Mumbai_Pune.2 = 0.0
Mumbai_Kolkata = 0.0
Mumbai_Kolkata.1 = 0.0
Mumbai_Unnamed: 1 = 0.0
Mumbai_Bihar = 0.0
Mumbai_Bihar.1 = 0.0
Mumbai_Bihar.2 = 0.0
Mumbai_Delhi = 0.0
Mumbai_Delhi.1 = 38.0
Mumbai_Delhi.2 = 0.0
Mumbai_Pune = 0.0
Mumbai_Pune.1 = 0.0
Mumbai_Pune.2 = 0.0
Mumbai_Kolkata = 0.0
Mumbai_Kolkata.1 = 2.0
Hyderabad_Unnamed: 1 = 5.0
Hyderabad_Bihar = 0.0
Hyderabad_Bihar.1 = 0.0
Hyderabad_Bihar.2 = 0.0
Hyderabad_Delhi = 0.0
Hyderabad_Delhi.1 = 0.0
Hyderabad_Delhi.2 = 0.0
Hyderabad_Pune = 30.0
Hyderabad_Pune.1 = 0.0
Hyderabad_Pu

In [36]:
supplier_data.columns

Index(['Bihar', 'Bihar.1', 'Bihar.2', 'Delhi', 'Delhi.1', 'Delhi.2', 'Pune',
       'Pune.1', 'Pune.2', 'Kolkata', 'Kolkata.1', 'Kolkata.2'],
      dtype='object')

In [42]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Read the input data
data = pd.read_excel(r"C:\Users\User\Downloads\archive\Travelling_3x3.xlsx", index_col=0)

# Extract data from the DataFrame
supply_cities = data.iloc[1:-1, 1:-1]
demand_cities = data.iloc[-1:, 1:-1]
inventory = data.iloc[1:-1, -1].values
demand = data.iloc[-1, 1:-1].values
costs = data.iloc[1:-1, 1:-1]

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define decision variables
# x = LpVariable.dicts("x", ((i, j) for i in supply_cities.index for j in demand_cities.columns), lowBound=0)
# Define decision variables using a normal for loop
x = {}
for i in supply_cities.index:
    for j in demand_cities.columns:
        x[(i, j)] = LpVariable(f"x_{i}_{j}", lowBound=0)

print ("this is X",x)
# Define the objective function
prob += lpSum(costs.at[i, j] * x[(i, j)] for i in supply_cities.index for j in demand_cities.columns)
print ("This is Problem",prob)

# Define supply constraints
# Define supply constraints
# Define supply constraints
for i, inv in enumerate(inventory):
    for j in demand_cities.columns:
        prob += lpSum(x[(i, j)] for i in supply_cities.index) <= inv

# Define demand constraints
for j in demand_cities.columns:
    prob += lpSum(x[(i, j)] for i in supply_cities.index) == demand_cities[j].iloc[0]

# Solve the problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Optimal Solution:")
for i in supply_cities.index:
    for j in demand_cities.columns:
        if x[(i, j)].varValue > 0:
            print(f"{i} to {j} = {x[(i, j)].varValue}")

print("\nTotal Cost =", round(prob.objective.value(), 2))


this is X {('Mumbai', 'Bihar'): x_Mumbai_Bihar, ('Mumbai', 'Bihar.1'): x_Mumbai_Bihar.1, ('Mumbai', 'Bihar.2'): x_Mumbai_Bihar.2, ('Mumbai', 'Delhi'): x_Mumbai_Delhi, ('Mumbai', 'Delhi.1'): x_Mumbai_Delhi.1, ('Mumbai', 'Delhi.2'): x_Mumbai_Delhi.2, ('Mumbai', 'Pune'): x_Mumbai_Pune, ('Mumbai', 'Pune.1'): x_Mumbai_Pune.1, ('Mumbai', 'Pune.2'): x_Mumbai_Pune.2, ('Mumbai', 'Kolkata'): x_Mumbai_Kolkata, ('Mumbai', 'Kolkata.1'): x_Mumbai_Kolkata.1, ('Mumbai', 'Kolkata.2'): x_Mumbai_Kolkata.2, ('Hyderabad', 'Bihar'): x_Hyderabad_Bihar, ('Hyderabad', 'Bihar.1'): x_Hyderabad_Bihar.1, ('Hyderabad', 'Bihar.2'): x_Hyderabad_Bihar.2, ('Hyderabad', 'Delhi'): x_Hyderabad_Delhi, ('Hyderabad', 'Delhi.1'): x_Hyderabad_Delhi.1, ('Hyderabad', 'Delhi.2'): x_Hyderabad_Delhi.2, ('Hyderabad', 'Pune'): x_Hyderabad_Pune, ('Hyderabad', 'Pune.1'): x_Hyderabad_Pune.1, ('Hyderabad', 'Pune.2'): x_Hyderabad_Pune.2, ('Hyderabad', 'Kolkata'): x_Hyderabad_Kolkata, ('Hyderabad', 'Kolkata.1'): x_Hyderabad_Kolkata.1, ('Hy

In [45]:
demand_cities.columns

Index(['Bihar', 'Bihar.1', 'Bihar.2', 'Delhi', 'Delhi.1', 'Delhi.2', 'Pune',
       'Pune.1', 'Pune.2', 'Kolkata', 'Kolkata.1', 'Kolkata.2'],
      dtype='object')

In [31]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Read the input data
data = pd.read_excel(r"C:\Users\User\Downloads\archive\Travelling_3x3.xlsx", index_col=0)

# supply_cities = data.iloc[1:-1, 1:-1]
# demand_cities = data.iloc[-1:, 1:-1]
# inventory = data.iloc[1:-1, -1].values
# demand = data.iloc[-1, 1:-1].values
# costs = data.iloc[1:-1, 1:-1]

# Extract data from the DataFrame
supply_cities = data.iloc[1:-1, 1:-1]
demand_cities = data.iloc[-1:, 1:-1]
inventory = data.iloc[1:-1, -1].values
demand = data.iloc[-1, 1:-1].values
costs = data.iloc[1:-1, 1:-1]

# Define the problem
prob = LpProblem("TransportationProblem", LpMinimize)

# Define decision variables
x = LpVariable.dicts("x", ((i, j) for i in supply_cities.index for j in demand_cities.columns), lowBound=0)
# print ("This is X",x)
# Define the objective function
prob += lpSum(costs.at[i, j] * x[(i, j)] for i in supply_cities.index for j in demand_cities.columns)
# print ("This is prob",prob)
# Define supply constraints
# for i in supply_cities.index:
#     prob += lpSum(x[(i, j)] for j in demand_cities.columns) <= inventory[i]
# Define supply constraints
for i, inv in enumerate(inventory):
    prob += lpSum(x[(i, j)] for j in demand_cities.columns) <= inv


# Define demand constraints
for j in demand_cities.columns:
    prob += lpSum(x[(i, j)] for i in supply_cities.index) == demand_cities[j].iloc[0]


# Solve the problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Optimal Solution:")
for i in supply_cities.index:
    for j in demand_cities.columns:
        if x[(i, j)].varValue > 0:
            print(f"{i} to {j} = {x[(i, j)].varValue}")

print("\nTotal Cost =", round(prob.objective.value(), 2))


KeyError: (0, 'Bihar')

In [46]:
costs

,Bihar,Bihar.1,Bihar.2,Delhi,Delhi.1,Delhi.2,Pune,Pune.1,Pune.2,Kolkata,Kolkata.1,Kolkata.2
Mumbai,12,100000,100000,5,100000,100000,7,100000,100000,4,100000,100000
Mumbai,100000,12,100000,100000,5,100000,100000,7,100000,100000,4,100000
Mumbai,100000,100000,12,100000,100000,5,100000,100000,7,100000,100000,4
Hyderabad,6,100000,100000,8,100000,100000,4,0,100000,12,100000,100000
Hyderabad,100000,6,100000,100000,8,100000,100000,4,100000,100000,12,100000
Hyderabad,100000,100000,6,100000,100000,8,100000,100000,4,100000,100000,12
Madhya Pradesh,6,100000,100000,5,100000,100000,2,100000,100000,4,100000,100000
Madhya Pradesh,100000,6,100000,100000,5,100000,100000,2,100000,100000,4,100000
Madhya Pradesh,100000,100000,6,100000,100000,5,100000,100000,2,100000,100000,4
Telangana,8,100000,100000,10,100000,100000,2,100000,100000,8,100000,100000


In [47]:
import numpy as np
from scipy.optimize import linprog

# Demand and inventory data
demand = np.array([40, 30, 52, 30, 40, 38, 48, 52, 30, 52, 38, 40])
inventory = np.array([50, 40, 40, 35, 58, 45, 75, 72, 45, 40, 30, 30])

# Cost matrix
cost_matrix = np.array([
    [12, 100000, 100000, 5, 100000, 100000, 7, 100000, 100000, 4, 100000, 100000],
    [100000, 12, 100000, 100000, 5, 100000, 100000, 7, 100000, 100000, 4, 100000],
    [100000, 100000, 12, 100000, 100000, 5, 100000, 100000, 7, 100000, 100000, 4],
    [6, 100000, 100000, 8, 100000, 100000, 4, 0, 100000, 12, 100000, 100000],
    [100000, 6, 100000, 100000, 8, 100000, 100000, 4, 100000, 100000, 12, 100000],
    [100000, 100000, 6, 100000, 100000, 8, 100000, 100000, 4, 100000, 100000, 12],
    [6, 100000, 100000, 5, 100000, 100000, 2, 100000, 100000, 4, 100000, 100000],
    [100000, 6, 100000, 100000, 5, 100000, 100000, 2, 100000, 100000, 4, 100000],
    [100000, 100000, 6, 100000, 100000, 5, 100000, 100000, 2, 100000, 100000, 4],
    [8, 100000, 100000, 10, 100000, 100000, 2, 100000, 100000, 8, 100000, 100000],
    [100000, 8, 100000, 100000, 10, 100000, 100000, 2, 100000, 100000, 8, 100000],
    [100000, 100000, 8, 100000, 100000, 10, 100000, 100000, 2, 100000, 100000, 8]
])

# Convert the maximization problem to minimization
c = cost_matrix.flatten() * -1

# Coefficient matrix for equality constraints
A_eq = []
for i in range(len(demand)):
    row = [0] * (i * len(cost_matrix[0])) + cost_matrix[i].tolist() + [0] * (len(cost_matrix[0]) * (len(demand) - i - 1))
    A_eq.append(row)

A_eq = np.array(A_eq)

# Right-hand side of equality constraints
b_eq = demand

# Coefficient matrix for inequality constraints (inventory)
A_ub = []
for i in range(len(inventory)):
    row = [0] * (i * len(cost_matrix[0])) + [1] * len(cost_matrix[0]) + [0] * (len(cost_matrix[0]) * (len(inventory) - i - 1))
    A_ub.append(row)

A_ub = np.array(A_ub)

# Right-hand side of inequality constraints
b_ub = inventory

# Bounds for decision variables (0 <= x <= 1)
bounds = [(0, 1) for _ in range(len(c))]

# Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Reshape the result to match the original matrix shape
optimal_solution = result.x.reshape(cost_matrix.shape)

print("Optimal Solution (selected quantities):\n", optimal_solution)
print("Minimum Cost:", -result.fun)


Optimal Solution (selected quantities):
 [[0.      0.0004  0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.0003  0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.00052 0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.      0.0003  0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.0004  0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.00038 0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.      0.00048 0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.00052 0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.0003  0.      0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]
 [0.      0.00052 0.      0.      0.      0.      0.      0.      0.
  0.      0.      0.     ]